In [40]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer
import warnings; warnings.filterwarnings("ignore");
from sklearn.metrics import mean_squared_error, make_scorer
import re

stemmer = SnowballStemmer('english')
df_train = pd.read_csv('train.csv',encoding="ISO-8859-1")
df_test = pd.read_csv('test.csv',encoding="ISO-8859-1")
df_attr = pd.read_csv('attributes.csv')
df_pro_desc = pd.read_csv('product_descriptions.csv',encoding="ISO-8859-1")


num_train = df_train.shape[0]

In [41]:
#ADD two blcok

In [42]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
class regvalues(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, hidden):
        dropcols=['id','relevance','search_term','product_title','product_description','product_info','attr','brand']
        hidden = hidden.drop(dropcols,axis=1).values
        return hidden
    
class regvalues1(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, hidden):
        dropcols=['id','relevance','product_uid','search_term','product_title','product_description','product_info','attr','brand']
        hidden = hidden.drop(dropcols,axis=1).values
        return hidden

class txtcolumn(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, data_dict):
        return data_dict[self.key].apply(str)

In [44]:
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
df_brand[0:9]

,product_uid,brand
9,100001.0,Simpson Strong-Tie
37,100002.0,BEHR Premium Textured DeckOver
69,100003.0,STERLING
93,100004.0,Grape Solar
122,100005.0,Delta
163,100006.0,Whirlpool
204,100007.0,Lithonia Lighting
236,100008.0,Teks
256,100009.0,House of Fara


In [45]:
collection = list(df_attr.name)
import re
from fuzzyfinder import fuzzyfinder
#suggestions = fuzzyfinder('material', L)
def fuzzyfinder(user_input, collection):
        suggestions =[]
        pattern = '.*'.join(user_input) # Converts 'djm' to 'd.*j.*m'
        regex = re.compile(pattern)     # Compiles a regex.
        for item in collection:
            match = regex.search(str(item))  # Checks if the current item matches the regex.
            if match:
                #suggestions.append((match.start(), item))
                suggestions.append(item)
        return suggestions
style_lower = fuzzyfinder('product style', collection)
style_upper = fuzzyfinder('Product Style', collection)
style_lower.extend(style_upper)

df_styles = df_attr[df_attr['name'].isin(style_lower)][["product_uid", "value"]].rename(columns={"value": "style"})
df_styles=df_styles.drop_duplicates(subset=['product_uid'], keep='first')
df_styles[1:5]

,product_uid,style
1784,100075.0,Cottage
2290,100092.0,Cottage
2936,100117.0,Classic
6259,100262.0,Modern


In [46]:

colour_lower = fuzzyfinder('Color Family', collection)
colour_upper = fuzzyfinder('Color Family', collection)
colour_lower.extend(colour_upper)

df_colours = df_attr[df_attr['name'].isin(colour_lower)][["product_uid", "value"]].rename(columns={"value": "colour"})
df_colours=df_colours.drop_duplicates(subset=['product_uid'], keep='first')
#df_colours[1:5]

In [47]:
df_type = fuzzyfinder('Type', collection)
df_type_= fuzzyfinder('type', collection)
df_type.extend(df_type_)

In [48]:
df_types = df_attr[df_attr['name'].isin(df_type)][["product_uid", "name"]].rename(columns={"name": "type"})
df_types=df_types.drop_duplicates(subset=['product_uid'], keep='first')
df_types[1:10]

,product_uid,type
66,100003.0,Installation Type
90,100004.0,Electrical Product Type
107,100005.0,Bath Faucet Type
132,100006.0,Appliance Type
180,100007.0,Battery Power Type
227,100008.0,Fastener Thread Type
259,100009.0,Moulding Type
281,100010.0,Landscape Supply Type
309,100011.0,Drive type


In [49]:

shape_lower = fuzzyfinder('shape', collection)
shape_upper = fuzzyfinder('Shape', collection)
shape_lower.extend(shape_upper)

df_shapes = df_attr[df_attr['name'].isin(shape_lower)][["product_uid", "value"]].rename(columns={"value": "shape"})
df_shapes=df_shapes.drop_duplicates(subset=['product_uid'], keep='first')
df_shapes[1:5]

,product_uid,shape
130,100005.0,Fixed Mount
227,100008.0,Standard
444,100016.0,Rectangular
666,100026.0,Square


# Merge data

In [50]:
num_train = df_train.shape[0] #74067
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True) #train+test zongxiangdiejia 240760
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid') #240760
df_all = pd.merge(df_all, df_styles, how='left', on='product_uid')
#df_all = pd.merge(df_all, df_function, how='left', on='product_uid')
df_all[0:5]

,id,product_title,product_uid,relevance,search_term,product_description,brand,style
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie,NaN
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie,NaN
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver,Cottage
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,Update your bathroom with the Delta Vero Singl...,Delta,NaN
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,Update your bathroom with the Delta Vero Singl...,Delta,NaN


In [51]:
df_all = pd.merge(df_all, df_colours, how='left', on='product_uid')

In [52]:
df_all = pd.merge(df_all, df_types, how='left', on='product_uid')

In [53]:
df_all = pd.merge(df_all, df_shapes, how='left', on='product_uid')

In [54]:
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}

def str_stem(s): 
    if isinstance(s, str):
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        s = s.replace("  "," ")
        s = s.replace(",","") #could be number / segment later
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = s.replace(" x "," xbi ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        s = s.replace("  "," ")
        s = s.replace(" . "," ")
        #s = (" ").join([z for z in s.split(" ") if z not in stop_w])
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        
        s = s.lower()
        s = s.replace("toliet","toilet")
        s = s.replace("airconditioner","air conditioner")
        s = s.replace("vinal","vinyl")
        s = s.replace("vynal","vinyl")
        s = s.replace("skill","skil")
        s = s.replace("snowbl","snow bl")
        s = s.replace("plexigla","plexi gla")
        s = s.replace("rustoleum","rust-oleum")
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless","whirlpool stainless")
        return s
    else:
        return "null"

In [55]:
def str_common_word(str1, str2):
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

In [56]:
def str_whole_word(str1, str2, i_):
    cnt = 0
    while i_ < len(str2):
        i_ = str2.find(str1, i_) #str1=search term, str2=product_title
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(str1)
    return cnt

# Need to process new feature

In [57]:
df_all['shape'] = df_all['shape'].map(lambda x:str_stem(x)) #map(function, sequence)

In [58]:
df_all['type'] = df_all['type'].map(lambda x:str_stem(x)) #map(function, sequence)

In [59]:
df_all['colour'] = df_all['colour'].map(lambda x:str_stem(x)) #map(function, sequence)

In [60]:
df_all['style'] = df_all['style'].map(lambda x:str_stem(x)) #map(function, sequence)

In [61]:
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stem(x)) #map(function, sequence)
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))
df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))
#df_all['type'] = df_all['type'].map(lambda x:str_stem(x))

In [62]:
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_types'] = df_all['type'].map(lambda x:len(x.split())).astype(np.int64)
#df_all['len_of_function_description'] = df_all['function_description'].map(lambda x:len(x.split())).astype(np.int64)
#df_all['search_term'] = df_all['product_info'].map(lambda x:seg_words(x.split('\t')[0],x.split('\t')[1]))
df_all[0:5]

,id,product_title,product_uid,relevance,search_term,product_description,brand,style,colour,type,shape,product_info,len_of_query,len_of_title,len_of_description,len_of_brand,len_of_types
0,2,simpson strong tie 12 gaug angl,100001,3.00,angl bracket,not onli do angl make joint stronger they also...,simpson strong tie,null,null,null,null,angl bracket\tsimpson strong tie 12 gaug angl\...,2,6,135,3,1
1,3,simpson strong tie 12 gaug angl,100001,2.50,l bracket,not onli do angl make joint stronger they also...,simpson strong tie,null,null,null,null,l bracket\tsimpson strong tie 12 gaug angl\tno...,2,6,135,3,1
2,9,behr premium textur deckov 1gal. #sc 141 tugbo...,100002,3.00,deck over,behr premium textur deckov is an innov solid c...,behr premium textur deckov,cottag,brown tan,paint product type,null,deck over\tbehr premium textur deckov 1gal. #s...,2,12,169,4,3
3,16,delta vero 1 handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat your bathroom with the delta vero singl ...,delta,null,chrome,bath faucet type,fix mount,rain shower head\tdelta vero 1 handl shower on...,3,14,109,1,3
4,17,delta vero 1 handl shower onli faucet trim kit...,100005,2.67,shower onli faucet,updat your bathroom with the delta vero singl ...,delta,null,chrome,bath faucet type,fix mount,shower onli faucet\tdelta vero 1 handl shower ...,3,14,109,1,3


In [63]:
df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['query_last_word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[1]))
df_all['query_last_word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[2]))

df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))

In [64]:
df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']
df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']+"\t"+df_all['type']
#df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']+"\t"+df_all['material']+"\t"+df_all['function_description']+"\t"+df_all['type']
df_all[0:5]

,id,product_title,product_uid,relevance,search_term,product_description,brand,style,colour,type,...,len_of_types,query_in_title,query_in_description,query_last_word_in_title,query_last_word_in_description,word_in_title,word_in_description,ratio_title,ratio_description,attr
0,2,simpson strong tie 12 gaug angl,100001,3.00,angl bracket,not onli do angl make joint stronger they also...,simpson strong tie,null,null,null,...,1,1,1,0,0,1,1,0.500000,0.500000,angl bracket\tsimpson strong tie\tnull
1,3,simpson strong tie 12 gaug angl,100001,2.50,l bracket,not onli do angl make joint stronger they also...,simpson strong tie,null,null,null,...,1,1,1,0,0,1,1,0.500000,0.500000,l bracket\tsimpson strong tie\tnull
2,9,behr premium textur deckov 1gal. #sc 141 tugbo...,100002,3.00,deck over,behr premium textur deckov is an innov solid c...,behr premium textur deckov,cottag,brown tan,paint product type,...,3,1,1,0,0,1,1,0.500000,0.500000,deck over\tbehr premium textur deckov\tpaint p...
3,16,delta vero 1 handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat your bathroom with the delta vero singl ...,delta,null,chrome,bath faucet type,...,3,1,1,0,0,1,1,0.333333,0.333333,rain shower head\tdelta\tbath faucet type
4,17,delta vero 1 handl shower onli faucet trim kit...,100005,2.67,shower onli faucet,updat your bathroom with the delta vero singl ...,delta,null,chrome,bath faucet type,...,3,3,3,1,1,3,3,1.000000,1.000000,shower onli faucet\tdelta\tbath faucet type


In [65]:
df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))#search_term, brand
df_all['word_in_type'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))#search_term, brand
df_all['ratio_brand'] = df_all['word_in_brand']/df_all['len_of_brand']
df_all['ratio_type'] = df_all['word_in_type']/df_all['len_of_types']

df_brand = pd.unique(df_all.brand.ravel()) #brand变为array后挑出唯一的brand name
#df_materials = pd.unique(df_all.material.ravel())
#df_function = pd.unique(df_all.function_description.ravel())
#df_materials = pd.unique(df_all.material.ravel())
df_all[0:2]

,id,product_title,product_uid,relevance,search_term,product_description,brand,style,colour,type,...,query_last_word_in_description,word_in_title,word_in_description,ratio_title,ratio_description,attr,word_in_brand,word_in_type,ratio_brand,ratio_type
0,2,simpson strong tie 12 gaug angl,100001,3.0,angl bracket,not onli do angl make joint stronger they also...,simpson strong tie,null,null,null,...,0,1,1,0.5,0.5,angl bracket\tsimpson strong tie\tnull,0,0,0.0,0.0
1,3,simpson strong tie 12 gaug angl,100001,2.5,l bracket,not onli do angl make joint stronger they also...,simpson strong tie,null,null,null,...,0,1,1,0.5,0.5,l bracket\tsimpson strong tie\tnull,0,1,0.0,1.0


In [66]:
d={}
i = 1000
for s in df_brand:
    #print(s)
    d[s]=i
    i+=3
#print(d)
df_all['brand_feature'] = df_all['brand'].map(lambda x:d[x])
df_all['search_term_feature'] = df_all['search_term'].map(lambda x:len(x)) #len(x): include the space
df_all[0:2]

,id,product_title,product_uid,relevance,search_term,product_description,brand,style,colour,type,...,word_in_description,ratio_title,ratio_description,attr,word_in_brand,word_in_type,ratio_brand,ratio_type,brand_feature,search_term_feature
0,2,simpson strong tie 12 gaug angl,100001,3.0,angl bracket,not onli do angl make joint stronger they also...,simpson strong tie,null,null,null,...,1,0.5,0.5,angl bracket\tsimpson strong tie\tnull,0,0,0.0,0.0,1000,12
1,3,simpson strong tie 12 gaug angl,100001,2.5,l bracket,not onli do angl make joint stronger they also...,simpson strong tie,null,null,null,...,1,0.5,0.5,l bracket\tsimpson strong tie\tnull,0,1,0.0,1.0,1000,9


# select feature

In [67]:
from sklearn import preprocessing
#Set the random seed
np.random.seed(12)
# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

#encode feature
df_all["style"] = label_encoder.fit_transform(df_all["style"])


In [68]:
#df_all["depth"] = label_encoder.fit_transform(df_all["depth"])

In [69]:
df_all["colour"] = label_encoder.fit_transform(df_all["colour"])

In [70]:
df_all["type"] = label_encoder.fit_transform(df_all["type"])

In [71]:
df_all["shape"] = label_encoder.fit_transform(df_all["shape"])

In [72]:
#df_all["material"] = label_encoder.fit_transform(df_all["material"])

In [73]:
#df_all = df_all.drop(['depth'],axis=1)

In [74]:
#df_all = df_all.drop(['material'],axis=1)

In [75]:
df_all = df_all.drop(['search_term','product_title','product_description','product_info','attr','brand'],axis=1)

In [76]:
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']

y_train = df_train['relevance'].values

X_train = df_train[:]
X_train = df_train.drop(['id','relevance'],axis=1).values

X_test = df_test[:]
X_test = df_test.drop(['id','relevance'],axis=1).values
#X_train = df_train[:]
#X_test = df_test[:]

In [77]:
X_train[1:5]

array([[  1.00001000e+05,   5.00000000e+00,   7.70000000e+01,
          3.03000000e+02,   1.53000000e+02,   2.00000000e+00,
          6.00000000e+00,   1.35000000e+02,   3.00000000e+00,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   5.00000000e-01,   5.00000000e-01,
          0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   1.00000000e+03,   9.00000000e+00],
       [  1.00002000e+05,   2.00000000e+00,   2.70000000e+01,
          3.15000000e+02,   1.53000000e+02,   2.00000000e+00,
          1.20000000e+01,   1.69000000e+02,   4.00000000e+00,
          3.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   5.00000000e-01,   5.00000000e-01,
          1.00000000e+00,   0.00000000e+00,   2.50000000e-01,
          0.00000000e+00,   1.00300000e+03,   9.00000000e+00],
      

In [78]:
#ADD two block

# train the model

In [79]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(X_train, y_train)
y_pred = svr_rbf.predict(X_test)
y_pred

array([ 2.60051686,  2.53357658,  2.5481506 , ...,  2.37679793,
        2.37679793,  2.37679793])

In [80]:
df_sol = pd.read_csv('solution.csv', encoding="ISO-8859-1")
df_sol['pred'] = y_pred
df_fliter = df_sol[df_sol.Usage == "Public"]

# evaluation

In [81]:
from sklearn.metrics import mean_squared_error, make_scorer
mse = mean_squared_error(df_fliter.relevance, df_fliter.pred, sample_weight=None, multioutput='uniform_average')
np.sqrt(mse)

0.53821779289788529

In [82]:
from sklearn.metrics import r2_score
R2 = r2_score(df_fliter.relevance, df_fliter.pred)  
print(R2)

-0.00984773402818


In [ ]:
#from sklearn.model_selection import cross_val_score
#score_rf = cross_val_score(clf,X_train,y_train,cv=5,scoring='mean_squared_error')
#score_nn = cross_val_score(nn,X_train,y_train,cv=5,scoring='mean_squared_error')
#score = 0.5*(score_rf + score_nn)
#score.mean()